In [15]:
import numpy as np
import pandas as pd

import category_encoders as ce

from collections import Counter

In [3]:
df_test = pd.read_csv('spatiotemporal_tst_data.csv')

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,11,13,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_test

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,SOURCE,HourlyAltimeterSetting,HourlyDewPointTemperature,...,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,BackupEquipment,BackupLatitude,BackupLongitude,BackupName,WindEquipmentChangeDate
0,42369099999,1999-03-12T22:00:00,26.760594,80.889339,124.96,"LUCKNOW, IN",FM-15,4,29.77,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42369099999,2007-03-31T05:30:00,26.760594,80.889339,124.96,"LUCKNOW, IN",FM-12,4,NaN,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42260099999,1977-02-19T20:30:00,27.155831,77.960892,167.94,"AGRA, IN",FM-12,4,NaN,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42634099999,1975-01-22T23:30:00,23.287828,69.670147,81.68,"BHUJ, IN",FM-12,4,NaN,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42170099999,2019-06-25T20:30:00,28.250000,74.916667,291.00,"CHURU, IN",FM-12,4,NaN,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103863,42379099999,2012-10-15T17:30:00,26.739708,83.449708,78.94,"GORAKHPUR, IN",FM-12,4,NaN,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103864,42273099999,1994-06-01T17:30:00,27.566667,81.600000,124.00,"BAHRAICH, IN",FM-12,4,NaN,68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103865,42452099999,1990-12-29T08:30:00,25.160219,75.845631,273.10,"KOTA, IN",FM-12,4,NaN,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103866,42503099999,2020-03-13T14:30:00,25.033333,88.133333,31.00,"MALDA, IN",FM-12,4,NaN,70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_miss = df_test.isna().sum(axis = 0)
df_miss

STATION                         0
DATE                            0
LATITUDE                        0
LONGITUDE                       0
ELEVATION                       0
                            ...  
BackupEquipment            103868
BackupLatitude             103868
BackupLongitude            103868
BackupName                 103868
WindEquipmentChangeDate    103868
Length: 122, dtype: int64

In [8]:
columns = ['STATION', 'LATITUDE', 'LONGITUDE', 'ELEVATION',
           'HourlyAltimeterSetting', 'HourlyDewPointTemperature',
           'HourlyDryBulbTemperature', 'HourlyPressureTendency',
           'HourlyRelativeHumidity', 'HourlySeaLevelPressure',
           'HourlyWindDirection', 'HourlyWindSpeed', 'HourlyPresentWeatherType',
           'HourlySkyConditions']

df_test = df_test.drop(columns=[col for col in df_test if col not in columns])
df_test

,STATION,LATITUDE,LONGITUDE,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySkyConditions,HourlySeaLevelPressure,HourlyWindDirection,HourlyWindSpeed
0,42369099999,26.760594,80.889339,124.96,29.77,50,73,||HZ,NaN,44.0,NaN,NaN,320,3
1,42369099999,26.760594,80.889339,124.96,NaN,64,72,||HZ,9.0,74.0,CLR:00,29.78,0,0
2,42260099999,27.155831,77.960892,167.94,NaN,57,72,||HZ,9.0,61.0,NaN,29.79,0,0
3,42634099999,23.287828,69.670147,81.68,NaN,52,66,NaN,NaN,60.0,NaN,29.99,270,5
4,42170099999,28.250000,74.916667,291.00,NaN,72,96,NaN,9.0,46.0,SCT:03 39,29.51,50,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103863,42379099999,26.739708,83.449708,78.94,NaN,74,81,||HZ,9.0,80.0,CLR:00,29.84,0,0
103864,42273099999,27.566667,81.600000,124.00,NaN,68,105,||DU,9.0,31.0,CLR:00,29.29,0,0
103865,42452099999,25.160219,75.845631,273.10,NaN,53,59,NaN,9.0,79.0,NaN,29.83,230,5
103866,42503099999,25.033333,88.133333,31.00,NaN,70,87,NaN,9.0,57.0,SCT:03 26,29.91,320,1


In [13]:
noise = []

for i in df_test.HourlyDewPointTemperature:
    try:
        float(i)
    except ValueError:
        noise.append(i)
print(noise)

[]


In [12]:
for j in ['HourlyAltimeterSetting', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature', 'HourlyPressureTendency', 
          'HourlyRelativeHumidity', 'HourlySeaLevelPressure', 'HourlyWindSpeed']:
    for i in range(0,len(df_test)):
        try:
            float(df_test[j][i])
        except ValueError:
            df_test[j][i] = df_test[j][i][:-1]
            float(df_test[j][i])

<ipython-input-12-af873a44c3e9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[j][i] = df_test[j][i][:-1]


### Encoding

In [ ]:
df_test_enc = df_test.copy()

In [16]:
#ENCODING HourlyPresentWeatherType

calzone = ['HourlyPresentWeatherType']

encoder= ce.BaseNEncoder(cols=calzone, return_df=True, base=6, handle_missing='return_nan')
data_encoded=encoder.fit_transform(df_test['HourlyPresentWeatherType'])
data_encoded

,HourlyPresentWeatherType_0,HourlyPresentWeatherType_1,HourlyPresentWeatherType_2
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
103863,0.0,0.0,1.0
103864,0.0,1.0,1.0
103865,NaN,NaN,NaN
103866,NaN,NaN,NaN


In [17]:
#ENCODING HourlySkyConditions
maximus = []
glad = []
for i in df_test['HourlySkyConditions']:
    if (':' in str(i)):
        count = Counter(i)
        #print(count)
        maximus.append(count[':'])
        sword = [j for j, ltr in enumerate(i) if ltr == ':']
        cus = 0
        for k in sword:
            cul = int(i[k+1:k+3])
            cus = cus + cul
        glad.append(cus)
        
        
win = int(max(maximus))
rem = win * 8

final = [item /rem for item in glad]

cup = []
#flag = 0
for x in df_test['HourlySkyConditions']:
    if pd.isna(x):
        cup.append(float(x))
    elif ':' not in x:
        cup.append(np.nan)
    else:
        cup.append(final.pop(0))
        #flag = flag + 1
        
HourlySkyConditions_01 = pd.DataFrame(cup)
HourlySkyConditions_01

,0
0,NaN
1,0.000
2,NaN
3,NaN
4,0.125
...,...
103863,0.000
103864,0.000
103865,NaN
103866,0.125


In [18]:
del df_test['HourlyPresentWeatherType']
del df_test['HourlySkyConditions']

df_test['HourlyPresentWeatherType_0'] = data_encoded['HourlyPresentWeatherType_0']
df_test['HourlyPresentWeatherType_1'] = data_encoded['HourlyPresentWeatherType_1']
df_test['HourlyPresentWeatherType_2'] = data_encoded['HourlyPresentWeatherType_2']
df_test['HourlySkyConditions_01'] = HourlySkyConditions_01[0]
df_test

,STATION,LATITUDE,LONGITUDE,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyWindDirection,HourlyWindSpeed,HourlyPresentWeatherType_0,HourlyPresentWeatherType_1,HourlyPresentWeatherType_2,HourlySkyConditions_01
0,42369099999,26.760594,80.889339,124.96,29.77,50,73,NaN,44.0,NaN,320,3,0.0,0.0,1.0,NaN
1,42369099999,26.760594,80.889339,124.96,NaN,64,72,9.0,74.0,29.78,0,0,0.0,0.0,1.0,0.000
2,42260099999,27.155831,77.960892,167.94,NaN,57,72,9.0,61.0,29.79,0,0,0.0,0.0,1.0,NaN
3,42634099999,23.287828,69.670147,81.68,NaN,52,66,NaN,60.0,29.99,270,5,NaN,NaN,NaN,NaN
4,42170099999,28.250000,74.916667,291.00,NaN,72,96,9.0,46.0,29.51,50,2,NaN,NaN,NaN,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103863,42379099999,26.739708,83.449708,78.94,NaN,74,81,9.0,80.0,29.84,0,0,0.0,0.0,1.0,0.000
103864,42273099999,27.566667,81.600000,124.00,NaN,68,105,9.0,31.0,29.29,0,0,0.0,1.0,1.0,0.000
103865,42452099999,25.160219,75.845631,273.10,NaN,53,59,9.0,79.0,29.83,230,5,NaN,NaN,NaN,NaN
103866,42503099999,25.033333,88.133333,31.00,NaN,70,87,9.0,57.0,29.91,320,1,NaN,NaN,NaN,0.125


In [23]:
noise = []

for i in df_test.HourlyWindDirection:
    try:
        float(i)
    except ValueError:
        noise.append(i)
print(noise)

['VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB', 'VRB'

In [22]:
for i in range(0,len(df_test)):
    if df_test['HourlyWindDirection'][i] != 'VRB':
        try:
            float(df_test['HourlyWindDirection'][i])
        except ValueError:
            df_test['HourlyWindDirection'][i] = df_test['HourlyWindDirection'][i][:-1]

In [24]:
megazord = []
for i in df_test.HourlyWindDirection:
    if (pd.isna(i))==False and i not in 'VRB':
        megazord.append(float(i))
superzord = pd.DataFrame(megazord)
superzord.describe()

,0
count,96079.000000
mean,141.602629
std,128.075014
min,0.000000
25%,0.000000
50%,110.000000
75%,270.000000
max,360.000000


In [25]:
np.mean(megazord)

141.6026290864809

In [26]:
for i in range(len(df_test.HourlyWindDirection)):
    if df_test.HourlyWindDirection[i] == 'VRB':
        df_test.HourlyWindDirection[i] = 141.602

df_test['HourlyWindDirection'] = df_test['HourlyWindDirection']

<ipython-input-26-aa069524404b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.HourlyWindDirection[i] = 141.602


In [27]:
df_test

,STATION,LATITUDE,LONGITUDE,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyWindDirection,HourlyWindSpeed,HourlyPresentWeatherType_0,HourlyPresentWeatherType_1,HourlyPresentWeatherType_2,HourlySkyConditions_01
0,42369099999,26.760594,80.889339,124.96,29.77,50,73,NaN,44.0,NaN,320,3,0.0,0.0,1.0,NaN
1,42369099999,26.760594,80.889339,124.96,NaN,64,72,9.0,74.0,29.78,0,0,0.0,0.0,1.0,0.000
2,42260099999,27.155831,77.960892,167.94,NaN,57,72,9.0,61.0,29.79,0,0,0.0,0.0,1.0,NaN
3,42634099999,23.287828,69.670147,81.68,NaN,52,66,NaN,60.0,29.99,270,5,NaN,NaN,NaN,NaN
4,42170099999,28.250000,74.916667,291.00,NaN,72,96,9.0,46.0,29.51,50,2,NaN,NaN,NaN,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103863,42379099999,26.739708,83.449708,78.94,NaN,74,81,9.0,80.0,29.84,0,0,0.0,0.0,1.0,0.000
103864,42273099999,27.566667,81.600000,124.00,NaN,68,105,9.0,31.0,29.29,0,0,0.0,1.0,1.0,0.000
103865,42452099999,25.160219,75.845631,273.10,NaN,53,59,9.0,79.0,29.83,230,5,NaN,NaN,NaN,NaN
103866,42503099999,25.033333,88.133333,31.00,NaN,70,87,9.0,57.0,29.91,320,1,NaN,NaN,NaN,0.125


### Imputation

In [28]:
%%time
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
impute_it = IterativeImputer()
X_impute = impute_it.fit_transform(df_test)

CPU times: user 2.48 s, sys: 794 ms, total: 3.28 s
Wall time: 803 ms


In [30]:
cls = list(df_test.columns)
df_test = pd.DataFrame(X_impute, columns = cls)
df_test

,STATION,LATITUDE,LONGITUDE,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyWindDirection,HourlyWindSpeed,HourlyPresentWeatherType_0,HourlyPresentWeatherType_1,HourlyPresentWeatherType_2,HourlySkyConditions_01
0,4.236910e+10,26.760594,80.889339,124.96,29.770000,50.0,73.0,8.939129,44.0,29.89275,320.0,3.0,0.000000,0.000000,1.000000,0.108998
1,4.236910e+10,26.760594,80.889339,124.96,29.708211,64.0,72.0,9.000000,74.0,29.78000,0.0,0.0,0.000000,0.000000,1.000000,0.000000
2,4.226010e+10,27.155831,77.960892,167.94,29.648442,57.0,72.0,9.000000,61.0,29.79000,0.0,0.0,0.000000,0.000000,1.000000,0.144252
3,4.263410e+10,23.287828,69.670147,81.68,29.749978,52.0,66.0,8.941651,60.0,29.99000,270.0,5.0,-0.000294,0.156698,2.336347,0.056814
4,4.217010e+10,28.250000,74.916667,291.00,29.715646,72.0,96.0,9.000000,46.0,29.51000,50.0,2.0,0.000470,0.326007,1.189858,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103863,4.237910e+10,26.739708,83.449708,78.94,30.157215,74.0,81.0,9.000000,80.0,29.84000,0.0,0.0,0.000000,0.000000,1.000000,0.000000
103864,4.227310e+10,27.566667,81.600000,124.00,29.240286,68.0,105.0,9.000000,31.0,29.29000,0.0,0.0,0.000000,1.000000,1.000000,0.000000
103865,4.245210e+10,25.160219,75.845631,273.10,29.545013,53.0,59.0,9.000000,79.0,29.83000,230.0,5.0,0.000159,0.221835,2.265027,0.167293
103866,4.250310e+10,25.033333,88.133333,31.00,30.247354,70.0,87.0,9.000000,57.0,29.91000,320.0,1.0,0.000100,-0.106561,1.003280,0.125000


In [31]:
df_test.isna().sum(axis = 0)

STATION                       0
LATITUDE                      0
LONGITUDE                     0
ELEVATION                     0
HourlyAltimeterSetting        0
HourlyDewPointTemperature     0
HourlyDryBulbTemperature      0
HourlyPressureTendency        0
HourlyRelativeHumidity        0
HourlySeaLevelPressure        0
HourlyWindDirection           0
HourlyWindSpeed               0
HourlyPresentWeatherType_0    0
HourlyPresentWeatherType_1    0
HourlyPresentWeatherType_2    0
HourlySkyConditions_01        0
dtype: int64

In [76]:
df_testD = pd.read_csv('spatiotemporal_tst_data.csv')
df_testD = df_testD.sort_values(by='DATE')
DATE = df_testD['DATE']

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,11,13,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [77]:
DATE = DATE.reset_index(drop = True)
DATE

0         1942-08-24T13:30:00
1         1942-08-26T17:30:00
2         1942-08-29T10:30:00
3         1942-08-31T09:30:00
4         1942-08-31T15:30:00
                 ...         
103863    2021-01-20T11:30:00
103864    2021-01-20T16:00:00
103865    2021-01-20T17:30:00
103866    2021-01-20T17:30:00
103867    2021-01-20T20:30:00
Name: DATE, Length: 103868, dtype: object

In [78]:
#adding the date attribute in date-time format

DATE = pd.to_datetime(DATE, format='%Y-%m-%dT%H:%M:%S')
DATE

0        1942-08-24 13:30:00
1        1942-08-26 17:30:00
2        1942-08-29 10:30:00
3        1942-08-31 09:30:00
4        1942-08-31 15:30:00
                 ...        
103863   2021-01-20 11:30:00
103864   2021-01-20 16:00:00
103865   2021-01-20 17:30:00
103866   2021-01-20 17:30:00
103867   2021-01-20 20:30:00
Name: DATE, Length: 103868, dtype: datetime64[ns]

In [79]:
#df_test.insert(0, 'DATE', DATE)
df_test

,DATE,STATION,LATITUDE,LONGITUDE,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyWindDirection,HourlyWindSpeed,HourlyPresentWeatherType_0,HourlyPresentWeatherType_1,HourlyPresentWeatherType_2,HourlySkyConditions_01
0,1942-08-24 13:30:00,4.236910e+10,26.760594,80.889339,124.96,29.770000,50.0,73.0,8.939129,44.0,29.89275,320.0,3.0,0.000000,0.000000,1.000000,0.108998
1,1942-08-26 17:30:00,4.236910e+10,26.760594,80.889339,124.96,29.708211,64.0,72.0,9.000000,74.0,29.78000,0.0,0.0,0.000000,0.000000,1.000000,0.000000
2,1942-08-29 10:30:00,4.226010e+10,27.155831,77.960892,167.94,29.648442,57.0,72.0,9.000000,61.0,29.79000,0.0,0.0,0.000000,0.000000,1.000000,0.144252
3,1942-08-31 09:30:00,4.263410e+10,23.287828,69.670147,81.68,29.749978,52.0,66.0,8.941651,60.0,29.99000,270.0,5.0,-0.000294,0.156698,2.336347,0.056814
4,1942-08-31 15:30:00,4.217010e+10,28.250000,74.916667,291.00,29.715646,72.0,96.0,9.000000,46.0,29.51000,50.0,2.0,0.000470,0.326007,1.189858,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103863,2021-01-20 11:30:00,4.237910e+10,26.739708,83.449708,78.94,30.157215,74.0,81.0,9.000000,80.0,29.84000,0.0,0.0,0.000000,0.000000,1.000000,0.000000
103864,2021-01-20 16:00:00,4.227310e+10,27.566667,81.600000,124.00,29.240286,68.0,105.0,9.000000,31.0,29.29000,0.0,0.0,0.000000,1.000000,1.000000,0.000000
103865,2021-01-20 17:30:00,4.245210e+10,25.160219,75.845631,273.10,29.545013,53.0,59.0,9.000000,79.0,29.83000,230.0,5.0,0.000159,0.221835,2.265027,0.167293
103866,2021-01-20 17:30:00,4.250310e+10,25.033333,88.133333,31.00,30.247354,70.0,87.0,9.000000,57.0,29.91000,320.0,1.0,0.000100,-0.106561,1.003280,0.125000


In [83]:
df = pd.read_csv('spatiotemporal_trn_data.csv')
df = df.sort_values(by='DATE')
DATE_train = df['DATE']
DATE_train = pd.to_datetime(DATE_train, format='%Y-%m-%dT%H:%M:%S')

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,11,13,18,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [88]:
DATE_train = DATE_train.reset_index(drop = True)
DATE_train

0        1942-08-24 07:30:00
1        1942-08-24 08:30:00
2        1942-08-24 09:30:00
3        1942-08-24 10:30:00
4        1942-08-24 11:30:00
                 ...        
934802   2021-01-21 02:30:00
934803   2021-01-21 02:30:00
934804   2021-01-21 02:30:00
934805   2021-01-21 02:30:00
934806   2021-01-21 02:30:00
Name: DATE, Length: 934807, dtype: datetime64[ns]

In [89]:
df_lag = pd.read_csv('CompleteTrainWithLabels.csv')

In [90]:
df_lag.insert(0, 'DATE', DATE_train)
df_lag

,DATE,Unnamed: 0,STATION,LATITUDE,LONGITUDE,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPressureTendency,...,HourlyPresentWeatherType_2,HourlySkyConditions_01,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,Lag,Visibility
0,1942-08-24 07:30:00,0,4.226010e+10,27.155831,77.960892,167.94,29.761189,61.036871,77.635273,8.955846,...,2.633696,0.125000,NaN,NaN,NaN,NaN,NaN,NaN,3.599834,99.42
1,1942-08-24 08:30:00,1,4.226010e+10,27.155831,77.960892,167.94,29.761014,61.048269,77.641731,8.905910,...,2.630588,0.125000,99.42,NaN,NaN,NaN,NaN,NaN,99.420000,99.42
2,1942-08-24 09:30:00,2,4.226010e+10,27.155831,77.960892,167.94,29.761091,61.058783,77.668865,8.907341,...,2.635532,0.125000,99.42,99.42,NaN,NaN,NaN,NaN,99.420000,99.42
3,1942-08-24 10:30:00,3,4.226010e+10,27.155831,77.960892,167.94,29.760177,61.094268,77.771681,8.894411,...,2.613123,0.166667,99.42,99.42,99.42,NaN,NaN,NaN,99.420000,99.42
4,1942-08-24 11:30:00,4,4.226010e+10,27.155831,77.960892,167.94,29.757577,61.191604,78.074961,8.895569,...,2.548382,0.291667,99.42,99.42,99.42,99.42,NaN,NaN,99.420000,99.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934802,2021-01-21 02:30:00,934802,4.237910e+10,26.739708,83.449708,78.94,29.440579,48.000000,52.000000,9.000000,...,0.000000,0.375000,0.31,0.03,0.75,0.93,2.49,2.49,0.030000,0.03
934803,2021-01-21 02:30:00,934803,4.236910e+10,26.760594,80.889339,124.96,29.643249,51.000000,54.000000,9.000000,...,2.455749,0.224097,0.03,0.31,0.03,0.75,0.93,2.49,0.750000,0.62
934804,2021-01-21 02:30:00,934804,4.263410e+10,23.287828,69.670147,81.68,29.452033,44.000000,60.000000,9.000000,...,3.315513,0.011531,0.62,0.03,0.31,0.03,0.75,0.93,2.490000,2.49
934805,2021-01-21 02:30:00,934805,4.217010e+10,28.250000,74.916667,291.00,29.630459,41.000000,45.000000,9.000000,...,2.143169,0.167774,2.49,0.62,0.03,0.31,0.03,0.75,2.490000,2.49


In [105]:
from tqdm import tqdm

In [ ]:
cstat = []
Lag = []

for j in tqdm(range(len(df_test))):
   # print('j',j)
    cstat = []
    for i in range(len(df_lag)):
        if df_lag['STATION'][i] == df_test['STATION'][j] and df_lag['DATE'][i] < df_test['DATE'][j]:
           # print('i',i)
            cstat.append(i)
        if df_lag['DATE'][i] > df_test['DATE'][j]:
            break
    if len(cstat)==0:
        Lag.append(df_lag['Lag'][0])
    else:
        length = len(cstat) - 1
        Lag.append(cstat.pop(length))

df_test['Lag'] = pd.DataFrame(Lag)

  7%|▋         | 7418/103868 [18:55:46<5461:22:09, 203.85s/it] 